In [2]:
import BioSimSpace as bss
import os
import nglview as nv
import subprocess as sp

In [3]:
def change_barostat(system: bss._SireWrappers._system.System,
                    protocol: bss.Protocol._equilibration.Equilibration,
                    work_directory: str,
                    process_name: str) -> None:
    """
    Change barostat in .mdp file for NPT runs
    @param: system
    @param: protocol
    @param: work_directory
    @param: process name
    @return: None
    """
    try:
        process = bss.Process.Gromacs(system,
                                      protocol,
                                      name=process_name,
                                      work_dir=work_directory)
    except RuntimeError:

        with open(f"{work_directory}/{process_name}.mdp", "r") as mdp:
            lines = mdp.read()

        new_lines = lines.replace("pcoupl = berendsen", "pcoupl = C-rescale")
        with open(f"{work_directory}/{process_name}.mdp", "w") as mdp:
            mdp.write(new_lines)


In [4]:
def write_mdp(work_dir: str, process_name: str, custom_options=[], default=True):
    """
    Write an mdp file for minimisation
    """
    default_options = ["nstlog = 100",
                       "integrator = steep",
                       "nstcgsteep = 1000",
                       "nsteps = 2000",
                       "pbc = xyz",
                       "cutoff-scheme = Verlet",
                       "ns-type = grid",
                       "nstlist = 20",
                       "coulombtype = PME",
                       "rvdw = 1.0",
                       "rcoulomb = 1.0",
                       "DispCorr = EnerPres",
                       "vdwtype = Cut-off",
                       "refcoord-scaling = all"]
    if default: 
        # print("Writing default mdp file.")
        with open(f"{work_dir}/{process_name}.mdp", "w") as mdp:
            for opt in default_options:
                mdp.write(f"{opt}\n")
        if len(custom_options) > 0:
            # print("Appending custom options.")
            with open(f"{work_dir}/{process_name}.mdp", "a") as mdp:
                for opt in custom_options:
                    mdp.write(f"{opt}\n")
    elif not default and custom_options != []:
        # print("Writing custom mdp file.")
        with open(f"{work_dir}/{process_name}.mdp", "w") as mdp:
            for opt in custom_options:
                mdp.write(f"{opt}\n")
    else:
        print("Custom options not specified. Not writing mdp file.")



In [57]:
def write_script(work_dir: str, process_name: str, restrained=False, previous="") -> str:
    script_name = work_dir + "/" + process_name + ".sh"
    with open(script_name, "w") as script:
        script.write("#!/bin/bash \n")
        script.write(f"cd {work_dir} \n")
        if not restrained and previous == "":
            script.write(f"gmx grompp -f {process_name}.mdp -c {process_name}.gro -p {process_name}.top -o {process_name}.tpr \n")
        elif not restrained and previous != "":
            script.write(f"gmx grompp -f {process_name}.mdp -c {previous}.gro -p {process_name}.top -t {previous}.cpt -o {process_name}.tpr \n")
        elif restrained and previous != "":
            script.write(f"gmx grompp -f {process_name}.mdp -c {previous}.gro -r {previous}.gro -p {process_name}.top -t {previous}.cpt -o {process_name}.tpr \n") 
        script.write(f"nohup gmx mdrun -v -deffnm {process_name} -nt 1 -nb gpu > nohup_{process_name}.out &")
    os.system(f"chmod +x {script_name}")
    executable = f"sh {script_name}"
    return executable


In [58]:
minimisation_steps = 2000
picosecond = bss.Units.Time.picosecond
kelvin = bss.Units.Temperature.kelvin
atm = bss.Units.Pressure.atm
short_nvt_runtime = 5 * picosecond
nvt_runtime = 50 * picosecond
npt_runtime = 200 * picosecond



In [59]:
n_ligands = 2
for i in range(n_ligands):
    ligand_datafile = open("ligands.dat", "r")
    ligand_lines = ligand_datafile.readlines()
    ligand_name = ligand_lines[i].rstrip()
    os.system(f"mkdir -p ../runs/equilibration/ligands/{ligand_name}")
    ligand_work_dir = f"../runs/equilibration/ligands/{ligand_name}"
    solvated_ligand = bss.IO.readMolecules([f"../inputs/ligands/{ligand_name}_solv.prm7",
                                            f"../inputs/ligands/{ligand_name}_solv.rst7"])
    ligand_minimisation_protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
    ligand_minimisation_process = bss.Process.Gromacs(solvated_ligand,
                                                      ligand_minimisation_protocol,
                                                      name="min",
                                                      work_dir=ligand_work_dir)
    write_mdp(ligand_work_dir, "min")
    ligand_min_script = write_script(ligand_work_dir, "min")
    # minimised_ligand = bss.IO.readMolecules([f"{ligand_work_dir}/min.gro",
    #                                          f"{ligand_work_dir}/min.top"])

    # ligand_r_nvt_protocol = bss.Protocol.Equilibration(runtime=short_nvt_runtime,
    #                                                    temperature_start=0*kelvin,
    #                                                    temperature_end=300*kelvin,
    #                                                    restraint="all")
    # ligand_r_nvt_process = bss.Process.Gromacs(minimised_ligand,
    #                                            ligand_r_nvt_protocol,
    #                                            name="r_nvt",
    #                                            work_dir=ligand_work_dir)
    # ligand_r_nvt_script = write_script(ligand_work_dir, "r_nvt", restrained=True, previous="min")
    
    # restrained_nvt_ligand = bss.IO.readMolecules([f"{ligand_work_dir}/r_nvt.gro",
    #                                               f"{ligand_work_dir}/r_nvt.top"])
    # ligand_nvt_protocol = bss.Protocol.Equilibration(runtime=nvt_runtime,
    #                                                  temperature=300*kelvin)
    # ligand_nvt_process = bss.Process.Gromacs(restrained_nvt_ligand,
    #                                          ligand_nvt_protocol,
    #                                          name="nvt",
    #                                          work_dir=ligand_work_dir)
    # ligand_nvt_script = write_script(ligand_work_dir, "nvt", previous="r_nvt")

    # ligand_nvt = bss.IO.readMolecules([f"{ligand_work_dir}/nvt.gro",
    #                                    f"{ligand_work_dir}/nvt.top"])
    # ligand_r_npt_protocol = bss.Protocol.Equilibration(runtime=npt_runtime,
    #                                                    pressure=1*atm,
    #                                                    temperature=300*kelvin,
    #                                                    restraint="heavy")
    # change_barostat(ligand_nvt, ligand_r_npt_protocol, ligand_work_dir, "r_npt")
    # ligand_r_npt_script = write_script(ligand_work_dir, "r_npt", restrained=True, previous="nvt")

    # restrained_npt_ligand = bss.IO.readMolecules([f"{ligand_work_dir}/r_npt.gro",
    #                                               f"{ligand_work_dir}/r_npt.top"])
    # ligand_npt_protocol = bss.Protocol.Equilibration(runtime=npt_runtime,
    #                                                  pressure=1*atm,
    #                                                  temperature=300*kelvin)    
    # change_barostat(restrained_npt_ligand, ligand_npt_protocol, ligand_work_dir, "npt")
    # ligand_npt_script = write_script(ligand_work_dir, "npt", previous="r_npt")


In [62]:
sp.run(ligand_min_script, shell=True)

Setting the LD random seed to -150995605

Generated 136 of the 136 non-bonded parameter combinations

Generated 136 of the 136 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'MOL'

Excluding 3 bonded neighbours molecule type 'SOL'

Excluding 3 bonded neighbours molecule type 'NA'
Analysing residue names:
There are:     1      Other residues
There are:   673      Water residues
There are:     1        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.402 nm
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 24x24x24, spacing 0.115 0.115 0.115

Estimate for the relative computational load of the PME mesh part: 0.30

This run will generate roughly 0 Mb of data


CompletedProcess(args='sh ../runs/equilibration/ligands/ligand_10/min.sh', returncode=0)

In [ ]:
sp.run(ligand_nvt_script)


In [ ]:
sp.run(ligand_r_npt_script)


In [ ]:
sp.run(ligand_npt_script)